In [76]:
# Imports
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.utils import pad_sequences
from keras_preprocessing.text import Tokenizer
import os
import numpy as np

print(tf.config.list_physical_devices('GPU'))

# Conda activate tf-gpu to active the GPU envrioment

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [77]:
# Dataset Preparation


input_texts = []
output_texts = []


with open("Buttons/prompts.txt") as f:
  input_texts = f.read().splitlines()

with open("Buttons/code.txt") as f:
  output_texts = f.read().splitlines()


In [78]:
# Tokenizer

tokenizer = Tokenizer(num_words= 20000, oov_token="<OOV>", char_level=False,split=" ",lower=False,filters="")

# So the Tokenizer converts words to a Token ID
# We have two Tokenizers one for Input taking the Prompt in this Case: "Button, blue-500, round, "Click me" "
# And we have one for Outputing the Answer of the Model in this case <Button classname="" ... > and so on
# num words is the ammount of the Top words we keep for the Tokenizer

combined_texts = []
for prompt, code in zip(input_texts, output_texts):
    combined_text = "<Start> " + prompt + " <SEP> " + code + " <End>"
    combined_texts.append(combined_text)

tokenizer.fit_on_texts(combined_texts)
tokenizer.word_index = {"<PAD>": 0, **{k: v+1 for k, v in tokenizer.word_index.items()}}
tokenizer.index_word = {v: k for k, v in tokenizer.word_index.items()}

# Training the Tokenizer on the vocab and adding Start and End Tokens

sequences = tokenizer.texts_to_sequences(combined_texts)


# So the text_to_sequences is therefore to convert each word or also called Token to its corresponding ID
# In this Case we first train the Tokenizers to generate IDs, and than we let the Tokenizers generate IDs

max_seq_len = max(len(seq) for seq in sequences)

# These are the neccessary lengths so we can pad all sequences in a batch to have the same size this process is calling padding 
# So we get the max length and afterwards check if the length of sequence is long enough else we add zero's till the max_encoder / max_decoder length is reached

sequences = pad_sequences(sequences,maxlen=max_seq_len,padding='post',truncating="post")
print(max_seq_len)

vocab_size = len(tokenizer.word_index) + 1




33


In [79]:
embed_dim = 128       
num_heads = 4          
ff_dim = 512        
num_layers = 4

#3125/3125 [==============================] - 459s 147ms/step - loss: 0.7670 - accuracy: 0.8050 - val_loss: 0.8075 - val_accuracy: 0.7951 - lr: 0.0010


inputs = layers.Input(shape=(max_seq_len-1))

embeddinglayer = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim,mask_zero=True)(inputs)

positional_encoding = layers.Embedding(input_dim=max_seq_len-1,output_dim=embed_dim)(tf.range(start=0, limit=max_seq_len-1,delta=1))

x = embeddinglayer + positional_encoding

# Only Self attention no encoder decoder attention
for i in range(num_layers):
  self_attention = layers.MultiHeadAttention(
  num_heads = num_heads,
  key_dim= embed_dim // num_heads,
  dropout=0.1,
  name=f"masked_self_attention_{i}"
  )(
    query=x,
    value=x,
    key=x,
    use_causal_mask=True   
  )
    
  x1 = layers.LayerNormalization(epsilon=1e-6)(x + self_attention) # residual connection by addiding the attention scores to the previous ones plus semantic context

  ffn = tf.keras.Sequential([
    layers.Dense(ff_dim,activation="gelu"),
    layers.Dropout(0.1),
    layers.Dense(embed_dim)
  ], name=f"feed_forward_{i}")

  ffn_output = ffn(x1)

  x = layers.LayerNormalization(epsilon=1e-6)(ffn_output + x1)

outputs = layers.Dense(
  vocab_size,
  activation="softmax",
  name="decoder_output_dense"
)(x)


In [ ]:
model = keras.Model(inputs=inputs, outputs=outputs, name="decoder_only_transformer")

initial_learning_rate = 1e-4
lr_schedule = keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=1000,
    alpha=0.1
)



model.compile(
    optimizer="adam",
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

model.summary()


callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5,  
        restore_best_weights=True,
        verbose=1
    ),
    
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),
    
    keras.callbacks.ModelCheckpoint(
        filepath='best_decoder_model.keras',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    
    keras.callbacks.LambdaCallback(
        on_batch_end=lambda batch, logs: tf.clip_by_global_norm([v for v in model.trainable_variables], 1.0)
    )
]


X = sequences[:, :-1]  
y = sequences[:, 1:]   


batch_size = 256
validation_split = 0.2 

history = model.fit(
    X,
    y,
    batch_size=batch_size,
    epochs=20,
    validation_split=validation_split,
    callbacks=callbacks,
    verbose=1,
    shuffle=True
)

model.save("Decoder_Transforer.keras")


Model: "decoder_only_transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 32)]         0           []                               
                                                                                                  
 embedding_22 (Embedding)       (None, 32, 128)      2039552     ['input_12[0][0]']               
                                                                                                  
 tf.__operators__.add_140 (TFOp  (None, 32, 128)     0           ['embedding_22[0][0]']           
 Lambda)                                                                                          
                                                                                                  
 masked_self_attention_0 (Multi  (None, 32, 128)     66048       ['tf.__ope

In [ ]:
#Inference: Not Neccessary
# inputs_inf = layers.Input(shape=(max_seq_len))

# embeddinglayer_inf = layers.Embedding(input_dim=vocab_size,output_dim=embed_dim, mask_zero=True)(inputs_inf)

# positional_encoding_inf = layers.Embedding(input_dim=max_seq_len,output_dim=embed_dim,mask_zero=True)(tf.range(start=0, limit=max_seq_len,delta=1))

# x_inf = embeddinglayer_inf + positional_encoding_inf

# for i in range(num_layers):
#   self_attention_inf = layers.MultiHeadAttention(
#     num_heads=num_heads,
#     key_dim= embed_dim//num_heads,
#     dropout=0.1
#   )(
#     query= x_inf,
#     value=x_inf,
#     key=x_inf,
#     use_causal_mask=True   
#   )

#   x2_inf = layers.LayerNormalization(epsilon=1e-6)(x_inf + self_attention_inf)

#   ffn_inf = keras.Sequential([
#     layers.Dense(ff_dim,activation="gelu"),
#     layers.Dense(ff_dim)
#   ])

#   ffn_output_inf = ffn_inf(x2_inf)

#   x3_inf = layers.LayerNormalization(epsilon=1e-6)(ffn_output_inf + x2_inf)

#   outputs = layers.Dense(
#   vocab_size,
#   activation="softmax",
#   name="decoder_output_dense"
# )(x)


In [ ]:
# Test Model
prompt = "Can you create a Light Yellow button that displays 'Confirm'?"

input_sequence = tokenizer.texts_to_sequences([prompt])
max_seq_len = max_seq_len -1
input_sequence = pad_sequences(input_sequence,maxlen=max_seq_len,padding='post')

decoder_transformer = keras.models.load_model('best_decoder_model.keras')
current_sequence = input_sequence.copy()
for _ in range(max_seq_len):
    predictions = decoder_transformer.predict(current_sequence)
    
    next_token = np.argmax(predictions[0, -1, :])      
    if next_token == tokenizer.word_index["<End>"]:
        break

    current_sequence = np.append(current_sequence, [[next_token]], axis=1)
    if current_sequence.shape[1] > max_seq_len:
        current_sequence = current_sequence[:, 1:]

original_length = input_sequence.shape[1]
generated_tokens = current_sequence[0, original_length:]
decoded_words = [tokenizer.index_word.get(i, '') for i in generated_tokens]
decoded_sentence = ' '.join(decoded_words)
print(decoded_sentence + "sentence")

ValueError: in user code:

    File "c:\Users\Henri\miniconda3\envs\tf-gpu\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\Henri\miniconda3\envs\tf-gpu\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Henri\miniconda3\envs\tf-gpu\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\Henri\miniconda3\envs\tf-gpu\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "c:\Users\Henri\miniconda3\envs\tf-gpu\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Henri\miniconda3\envs\tf-gpu\lib\site-packages\keras\engine\input_spec.py", line 295